In [44]:
import os
import re
import json
results = {}
with open('./results.json', 'r', encoding='utf-8') as f:
    results = json.load(f)
    
codigos = results.keys()

In [46]:

# result = { 
#     "pontuacao" : None,
#     "n_subtarefas" : None,
#     "subtarefas100" : None,
#     "tempo_lim" : None,
#     "mem_lim" : None,
#     "tempo" : None,
#     "mem" : None,
#     "comp" : None,
#     "stderr" : None 
# }
list_error = []
list_warning = []
list_accepted = []
list_TLE = []
list_MLE = []
for codigo in codigos:
    data = results[codigo]
    if not data["comp"]:
        list_error.append(codigo)
        continue
    if len(data["stderr"])>1:
        list_warning.append(codigo)
    if data["pontuacao"] == 100:
        list_accepted.append(codigo)
    if data["tempo"] >= data["tempo_lim"]:
        list_TLE.append(codigo)
    if data["mem"] >= data["mem_lim"]:
        list_MLE.append(codigo)



    
        

In [47]:
def count_words(text):
    return len(re.findall(r'\S+', text))
def get_info(info, codigo):
    info[codigo] = {
        "len_enunciado": None,
        "len_entrada": None,
        "len_saida": None,
        "n_restricoes": None,
        "n_exemplos": None
    }
    text = ''
    with open(f'./pratique/{codigo}/{codigo}.txt', 'r', encoding='utf-8') as f:
        text = f.read()
    data = info[codigo]
    sections = re.split(r'\n\s*Entrada\s*\n',text)
    enunciado = ''.join(sections[:2])
    data["len_enunciado"] = count_words(enunciado)
    n_exemplos = len(sections)-2
    len_entrada = 0
    len_saida = 0
    if n_exemplos > 0:
        for section in sections [2:]:
            splitted = re.split(r'\n\s*Saída\s*\n',section) # verificado se existe string 'Saida' nos textos # verificado se as strings procuradas existem nos textos
            entrada = splitted[0]
            try:
                saida = splitted[1]
            except: # quebra em caso de entrada vazia
                print(f'{codigo} com entrada vazia.')
                return
            if len(entrada.strip()) == 0 or len(saida.strip()) == 0:
                print(f'{codigo} com entrada ou saida vazia.')
                return
            len_entrada += count_words(entrada)
            len_saida += count_words(saida)
        len_entrada /= n_exemplos
        len_saida /= n_exemplos
        data["len_entrada"] = len_entrada
        data["len_saida"] = len_saida
    data["n_exemplos"] = n_exemplos
    data["n_restricoes"] = enunciado.count('≤')+enunciado.count('≥')
info = {}
for codigo in codigos:
    get_info(info, codigo)

 
        

In [48]:
comentarios = {}
def get_comentarios(comentarios, codigo):
    with open(f'./pratique/{codigo}/{codigo}.cpp', 'r', encoding='utf-8') as f:
        code = f.read()
    search = re.findall(r'(//.*?$|/\*.*?\*/)', code, re.DOTALL | re.MULTILINE)
    if len(search) > 0:
        comentarios[codigo] = search
for codigo in codigos:
    get_comentarios(comentarios, codigo)


In [49]:
contain_image = []
with open('./pratique/contain_image.json', 'r', encoding='utf-8') as f:
    contain_image = json.load(f)

In [50]:
# Importando bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações globais para os gráficos
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["font.size"] = 12

# Convertendo os dicionários em DataFrames para facilitar a análise
results_df = pd.DataFrame.from_dict(results, orient="index")
info_df = pd.DataFrame.from_dict(info, orient="index")

# Unindo os dois DataFrames
data = results_df.join(info_df)

# Filtrando questões válidas (sem erro de compilação, TLE ou MLE)
valid_data = data[
    (data["comp"] == True) &
    (~data.index.isin(list_TLE)) &
    (~data.index.isin(list_MLE))
]



In [ ]:
# 1) Cumulative Distribution Function (CDF) das pontuações
plt.figure()
sns.ecdfplot(data=valid_data, x="pontuacao", stat="proportion")
plt.title("Função de Distribuição Acumulada das Pontuações")
plt.xlabel("Pontuação (%)")
plt.ylabel("Proporção Acumulada")
plt.tight_layout()
plt.show()

# 2) Porcentagens de acertos por quantidade de subtarefas
valid_data["subtask_group"] = pd.cut(
    valid_data["n_subtarefas"],
    bins=[0, 3, 6, 10, np.inf],
    labels=["1-3", "4-6", "7-10", "11+"]
)
valid_data["accuracy"] = valid_data["subtarefas100"] / valid_data["n_subtarefas"] * 100

plt.figure()
sns.boxplot(data=valid_data, x="subtask_group", y="accuracy", palette="Blues")
plt.title("Porcentagem de Acertos por Quantidade de Subtarefas")
plt.xlabel("Grupo de Subtarefas")
plt.ylabel("Porcentagem de Acertos (%)")
plt.tight_layout()
plt.show()

# 3) Comparativo entre questões com e sem imagens
data["has_image"] = data.index.isin(contain_image)
grouped_image = data.groupby("has_image").agg(
    pontuacao_media=("pontuacao", "mean"),
    erros=("comp", lambda x: (~x).sum()),
    TLE=("tempo", lambda x: x.isin(list_TLE).sum()),
    MLE=("mem", lambda x: x.isin(list_MLE).sum())
).reset_index()

plt.figure()
grouped_image.plot(kind="bar", x="has_image", stacked=True, color=["#4c72b0", "#55a868", "#c44e52"])
plt.title("Comparativo entre Questões com e sem Imagens")
plt.xlabel("Contém Imagem")
plt.ylabel("Média e Erros")
plt.tight_layout()
plt.show()

# 4) Pontuação média por edição, nível e fase
data["edition"] = data.index.str.split("_").str[0]
data["level"] = data.index.str.split("_").str[1]
data["phase"] = data.index.str.split("_").str[2]

grouped_edition = data.groupby(["edition", "level", "phase"])["pontuacao"].mean().unstack()

plt.figure()
grouped_edition.plot(kind="bar", cmap="viridis")
plt.title("Pontuação Média por Edição, Nível e Fase")
plt.xlabel("Edição, Nível e Fase")
plt.ylabel("Pontuação Média")
plt.tight_layout()
plt.show()

# 5) Porcentagem de TLE, MLE e erros de compilação por edição, nível e fase
verdict_counts = data.groupby(["edition", "level", "phase"]).apply(
    lambda x: pd.Series({
        "TLE": x.index.isin(list_TLE).sum(),
        "MLE": x.index.isin(list_MLE).sum(),
        "Erro de Compilação": (~x["comp"]).sum()
    })
).reset_index()

verdict_counts_melted = verdict_counts.melt(
    id_vars=["edition", "level", "phase"],
    value_vars=["TLE", "MLE", "Erro de Compilação"],
    var_name="Veredito",
    value_name="Quantidade"
)

plt.figure()
sns.barplot(data=verdict_counts_melted, x="edition", y="Quantidade", hue="Veredito", palette="Set2")
plt.title("Porcentagem de Vereditos por Edição, Nível e Fase")
plt.xlabel("Edição, Nível e Fase")
plt.ylabel("Porcentagem")
plt.tight_layout()
plt.show()

# 6) Percentual de pontuação 100 por edição, nível e fase
accepted_counts = data.groupby(["edition", "level", "phase"]).apply(
    lambda x: (x.index.isin(list_accepted).sum() / len(x)) * 100
).reset_index(name="Percentual de Pontuação 100")

plt.figure()
sns.barplot(data=accepted_counts, x="edition", y="Percentual de Pontuação 100", hue="level", palette="coolwarm")
plt.title("Percentual de Pontuação 100 por Edição, Nível e Fase")
plt.xlabel("Edição, Nível e Fase")
plt.ylabel("Percentual (%)")
plt.tight_layout()
plt.show()

# 7) Relação entre tamanho do enunciado/entradas/saídas e pontuações/vereditos
plt.figure()
sns.pairplot(data, vars=["len_enunciado", "len_entrada", "len_saida", "pontuacao"], hue="comp", palette="husl")
plt.suptitle("Relação entre Tamanho do Enunciado/Entradas/Saídas e Pontuações")
plt.tight_layout()
plt.show()